In [3]:
import pickle
from ai import use_key
from model import index_file

In [4]:
api_key = "API KEY"
use_key(api_key)

In [5]:
import pinecone
index_name = 'regulations'
pinecone.init(
    api_key= "pine cone api key",
    environment="us-west1-gcp"
)


C:\Users\mnili\Anaconda3\envs\ask-pdf\lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [6]:
index = pinecone.Index(index_name)

In [7]:
pinecone.list_indexes()

['regulations']

In [34]:
import os

folder_path = 'pdf'
paths =[]
# Loop through all the files in the folder
for filename in os.listdir(folder_path):
    # Check if the current file is a file or directory
    if os.path.isfile(os.path.join(folder_path, filename)):
        paths.append( folder_path+'/'+filename)

i=25
id=0
for x in range(i,len(paths)):
    i = i+1

    print(str(i)+'/'+str(len(paths)))
    pdf_name = paths[x][4:-4]
    print(pdf_name)
    print("file :"+paths[x])
    #indexing data
    out = index_file(paths[x])
    with open("pkl/"+pdf_name+".pkl", "wb") as f:
        pickle.dump(out, f)
    #creating list to send to pinecone with meta data
    upsert_input = []
    for k in range(len(out['vectors'])):
        id = id +1
        upsert_input.append((str(id),out['vectors'][k],{"page":str(k)}))

    #sending data to pinecone
    j = 0
    while j < len(upsert_input):
        print(str(j)+'/'+str(len(upsert_input)))
        if j+100<=len(upsert_input):
            index.upsert(vectors=upsert_input[j:j+100],namespace=pdf_name)
            j = j+100
        else:
            index.upsert(vectors=upsert_input[j:],namespace=pdf_name)
            j = len(upsert_input)
            break
    print(index.describe_index_stats())

26/33
DFARSPGI
file :pdf/DFARSPGI.pdf
vectorizing :0/598
vectorizing :1/598
vectorizing :2/598
vectorizing :3/598
vectorizing :4/598
vectorizing :5/598
vectorizing :6/598
vectorizing :7/598
vectorizing :8/598
vectorizing :9/598
vectorizing :10/598
vectorizing :11/598
vectorizing :12/598
vectorizing :13/598
vectorizing :14/598
vectorizing :15/598
vectorizing :16/598
vectorizing :17/598
vectorizing :18/598
vectorizing :19/598
vectorizing :20/598
vectorizing :21/598
vectorizing :22/598
vectorizing :23/598
vectorizing :24/598
vectorizing :25/598
vectorizing :26/598
vectorizing :27/598
vectorizing :28/598
vectorizing :29/598
vectorizing :30/598
vectorizing :31/598
vectorizing :32/598
vectorizing :33/598
vectorizing :34/598
vectorizing :35/598
vectorizing :36/598
vectorizing :37/598
vectorizing :38/598
vectorizing :39/598
vectorizing :40/598
vectorizing :41/598
vectorizing :42/598
vectorizing :43/598
vectorizing :44/598
vectorizing :45/598
vectorizing :46/598
vectorizing :47/598
vectorizing 

In [34]:
pdf_name = "vaar"
out = index_file("pdf/"+pdf_name+".pdf")
len(out)

vectorizing :0/152
vectorizing :1/152
vectorizing :2/152
vectorizing :3/152
vectorizing :4/152
vectorizing :5/152
vectorizing :6/152
vectorizing :7/152
vectorizing :8/152
vectorizing :9/152
vectorizing :10/152
vectorizing :11/152
vectorizing :12/152
vectorizing :13/152
vectorizing :14/152
vectorizing :15/152
vectorizing :16/152
vectorizing :17/152
vectorizing :18/152
vectorizing :19/152
vectorizing :20/152
vectorizing :21/152
vectorizing :22/152
vectorizing :23/152
vectorizing :24/152
vectorizing :25/152
vectorizing :26/152
vectorizing :27/152
vectorizing :28/152
vectorizing :29/152
vectorizing :30/152
vectorizing :31/152
vectorizing :32/152
vectorizing :33/152
vectorizing :34/152
vectorizing :35/152
vectorizing :36/152
vectorizing :37/152
vectorizing :38/152
vectorizing :39/152
vectorizing :40/152
vectorizing :41/152
vectorizing :42/152
vectorizing :43/152
vectorizing :44/152
vectorizing :45/152
vectorizing :46/152
vectorizing :47/152
vectorizing :48/152
vectorizing :49/152
vectorizin

5

In [35]:
with open("pkl/"+pdf_name+".pkl", "wb") as f:
    pickle.dump(out, f)

--------------------- WORKS but takes time so we will use pkl file ----------------------------

In [36]:
with open("pkl/"+pdf_name+".pkl", "rb") as f:
    out = pickle.load(f)


##Nombre de vecteur

In [37]:
len(out['vectors'])

152

Dimension des vecteurs

In [38]:
len(out['vectors'][0])

1536

In [39]:
for i in out:
    print(i)

size
texts
pages
vectors
summary


In [40]:
out['summary']

'This is a fragment from the Department of Veterans Affairs Acquisition Regulation System.'

In [10]:
import pinecone

C:\Users\mnili\Anaconda3\envs\ask-pdf\lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [17]:
pinecone.list_indexes()

[]

In [26]:
metadata_config = {
    "indexed": ["page"]
}

In [8]:
pinecone.describe_index(index_name)

IndexDescription(name='vaar', metric='cosine', replicas=1, dimension=1536.0, shards=1, pods=1, pod_type='p1.x1', status={'ready': False, 'state': 'Initializing'}, metadata_config=None, source_collection='')

In [25]:
pinecone.delete_index(index_name)

In [27]:
pinecone.create_index(index_name.lower(),dimension=1536,metric="cosine", metadata_config=metadata_config)

In [8]:
index = pinecone.Index("regulations")


In [41]:
upsert_input = []
for i in range(len(out['vectors'])):
    upsert_input.append((str(i),out['vectors'][i],{"regulation-pdf":pdf_name}))

In [42]:
i = 0
while i < len(upsert_input):
    print(str(i)+'/'+str(len(upsert_input)))
    if i+100<=len(upsert_input):
        index.upsert(upsert_input[i:i+100])
        i = i+100
    else:
        index.upsert(upsert_input[i:])
        i = len(upsert_input)
        break

0/152
100/152


In [9]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'AFARS_Full': {'vector_count': 197},
                'AFFARS': {'vector_count': 562},
                'CFR-2021-title48-vol4-chap3': {'vector_count': 71},
                'CFR-2021-title48-vol4-chap4': {'vector_count': 71},
                'CFR-2021-title48-vol4-chap6': {'vector_count': 99},
                'CFR-2021-title48-vol5-chap10': {'vector_count': 24},
                'CFR-2021-title48-vol5-chap12': {'vector_count': 63},
                'CFR-2021-title48-vol5-chap13': {'vector_count': 105},
                'CFR-2021-title48-vol5-chap14': {'vector_count': 78},
                'CFR-2021-title48-vol5-chap7': {'vector_count': 151},
                'CFR-2021-title48-vol5-chap8': {'vector_count': 152},
                'CFR-2021-title48-vol5-chap9': {'vector_count': 280},
                'CFR-2021-title48-vol6-chap15': {'vector_count': 110},
                'CFR-2021-title48-vol6-chap16': {'vector_count': 56},
               

In [22]:
from model import get_vectors
text = ["Deviations from the FAR or AIDAR "]
v = get_vectors(text)


vectorizing :0/1


In [21]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'AFARS_Full': {'vector_count': 197},
                'AFFARS': {'vector_count': 562},
                'CFR-2021-title48-vol4-chap3': {'vector_count': 71},
                'CFR-2021-title48-vol4-chap4': {'vector_count': 71},
                'CFR-2021-title48-vol4-chap6': {'vector_count': 99},
                'CFR-2021-title48-vol5-chap10': {'vector_count': 24},
                'CFR-2021-title48-vol5-chap12': {'vector_count': 63},
                'CFR-2021-title48-vol5-chap13': {'vector_count': 105},
                'CFR-2021-title48-vol5-chap14': {'vector_count': 78},
                'CFR-2021-title48-vol5-chap7': {'vector_count': 151},
                'CFR-2021-title48-vol5-chap8': {'vector_count': 152},
                'CFR-2021-title48-vol5-chap9': {'vector_count': 280},
                'CFR-2021-title48-vol6-chap15': {'vector_count': 110},
                'CFR-2021-title48-vol6-chap16': {'vector_count': 56},
               

In [33]:
response = index.query(
  vector=v,
  top_k=1,
    include_metadata =True,
    #include_values=True,
    namespace="TRANSFARS"
)



response

{'matches': [{'id': '3581',
              'metadata': {'page': '4'},
              'score': 0.818531275,
              'values': []}],
 'namespace': 'TRANSFARS'}

In [31]:
id = response['matches'][0]['id']
id

IndexError: list index out of range

In [1]:
from model import query2,get_response_vectors
import pickle
from ai import use_key
from model import index_file
import pinecone
from model import get_vectors

C:\Users\mnili\Anaconda3\envs\ask-pdf\lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
api_key = "api key"
use_key(api_key)
index_name = 'regulations'
pinecone.init(
    api_key="pinecone api key",
    environment="us-west1-gcp"
)
index = pinecone.Index(index_name)
text = "talk about Agency control and compliance procedures"
response_vector = get_response_vectors(['vaar','TRANSFARS','SOFARS Full'],[text],index )
response_query = query2(response_vector,text)

vectorizing :0/1


In [3]:
response_query['text']

'The Agency control and compliance procedures are described in section 5501.304 of the text. The following points summarize the key aspects of this section:\n\n- Requests for clauses must be submitted in writing to TCAQ-P and include a copy of the clause(s) and sufficient justification as to why the clause is needed.\n- The Head of the Contracting Activity is the approval authority for local clauses.\n- TCAQ-P conducts an annual review of all TRANSF ARS clauses in use, which may recommend retention, deletion, or modification of the existing clauses.\n- Changes and revisions to the TRANSF ARS, policy letters, and operating instructions are distributed to Directorate of Acquisition divisions electronically and posted to the shared drive.\n- All documents relating to procurement matters that require review, coordination, and/or approval by the Office of the Under Secretary of Defense, Defense Procurement and Acquisition Policy (DP AP) must be sent through TCAQ-P and be endorsed in writing